In [2]:
import os

# Set your desired directory path
download_path = '/Users/muhammadsohaib/Downloads/'

# Optional: Change working directory to the download path
os.chdir(download_path)


In [3]:
# d4d77f6b_morph_gradient_instances.nii.gz 
# f783490b_clean_reduced_instances.nii.gz

# da2c0d48_clean_reduced_instances.nii.gz
## e8ace21c_clean_reduced_instances.nii.gz
#3284319f_morph_gradient_instances.nii.gz


In [5]:
import cc3d
import numpy as np
import vtk
from vtk.util import numpy_support
import SimpleITK as sitk
import nibabel as nib

# Load your segmentation data
nii_path = "da2c0d48_clean_reduced_instances.nii.gz"
img = nib.load(nii_path)
labels_in = img.get_fdata()

# Ensure that labels are consecutive integers starting from 1
labels_out = cc3d.connected_components(labels_in, connectivity=26)  # 6-connected

# Create a VTK image data object
vtk_image = vtk.vtkImageData()
vtk_image.SetDimensions(labels_out.shape[::-1])  # VTK uses a different order (z, y, x)
vtk_image.AllocateScalars(vtk.VTK_UNSIGNED_CHAR, 1)

# Copy NumPy array data to VTK image
vtk_array = numpy_support.numpy_to_vtk(labels_out.ravel(), deep=True)
vtk_image.GetPointData().SetScalars(vtk_array)

# Create a VTK Discrete Marching Cubes algorithm
discrete_marching_cubes = vtk.vtkDiscreteMarchingCubes()
discrete_marching_cubes.SetInputData(vtk_image)
discrete_marching_cubes.GenerateValues(np.max(labels_out), 1, np.max(labels_out))

# Apply smoothing
smooth_filter = vtk.vtkSmoothPolyDataFilter()
smooth_filter.SetInputConnection(discrete_marching_cubes.GetOutputPort())
smooth_filter.SetNumberOfIterations(2000)  # Adjust the number of iterations as needed: sMOOTH

# Create a VTK renderer and render window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.SetWindowName("Connected Components 3D Visualization")
render_window.SetSize(800, 800)
render_window.AddRenderer(renderer)

# Create a VTK render window interactor
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# Create a VTK lookup table
lut = vtk.vtkLookupTable()
lut.SetNumberOfTableValues(np.max(labels_out) + 1)
lut.Build()

# Assign different colors to each connected component label
for i in range(1, np.max(labels_out) + 1):
    color = np.random.rand(3)  # Random RGB color
    lut.SetTableValue(i, color[0], color[1], color[2])

# Create a VTK PolyDataMapper
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(smooth_filter.GetOutputPort())  # Use the smoothed output
mapper.SetLookupTable(lut)
mapper.SetScalarRange(0, np.max(labels_out))

# Create a VTK actor
actor = vtk.vtkActor()
actor.SetMapper(mapper)

# Add the actor to the renderer
renderer.AddActor(actor)

# Set up the renderer
renderer.SetBackground(255, 255, 255)  # Light gray background

camera = renderer.GetActiveCamera()
camera.SetPosition(0, 0, 500)  # Position the camera (adjust as needed)
camera.SetFocalPoint(700, 0, 400)  # Set the focal point of the camera
camera.SetViewUp(0, 1, 0)  # Define the upward direction of the camera
renderer.ResetCamera()

# Add axes to the scene
axes_actor = vtk.vtkAxesActor()
axes_widget = vtk.vtkOrientationMarkerWidget()
axes_widget.SetOrientationMarker(axes_actor)
axes_widget.SetInteractor(render_window_interactor)
axes_widget.SetViewport(0.1, 0.0, 1.0, 0.2)  # Adjust the viewport position and size

# Start the rendering loop
render_window.Render()
render_window_interactor.Start()


In [ ]:
import os
directory_path = os.getcwd()


In [1]:
import os
import numpy as np
import SimpleITK as sitk

# Use the current working directory
directory_path = os.getcwd()

# Get all NIfTI files in the directory
nii_files = [f for f in os.listdir(directory_path) if f.endswith(".nii.gz")]

# Initialize global min and max intensity values
global_min_intensity = float('inf')
global_max_intensity = float('-inf')

# Process each file
for file in nii_files:
    try:
        print(f"\n🔹 Processing file: {file}")
        full_path = os.path.join(directory_path, file)

        # Read the NIfTI image
        image = sitk.ReadImage(full_path)

        # Get spacing and shape
        spacing = image.GetSpacing()
        shape = image.GetSize()

        # Convert image to numpy array
        image_array = sitk.GetArrayFromImage(image)

        # Get min/max intensity values for the current image
        min_intensity = np.min(image_array)
        max_intensity = np.max(image_array)

        # Update global min and max intensity
        global_min_intensity = min(global_min_intensity, min_intensity)
        global_max_intensity = max(global_max_intensity, max_intensity)

        # Print details
        print(f"   📏 Spacing: {spacing}")
        print(f"   📐 Shape: {shape} (width, height, depth)")
        print(f"   📦 Image array shape: {image_array.shape} (depth, height, width)")
        print(f"   🔽 Min intensity value: {min_intensity}")
        print(f"   🔼 Max intensity value: {max_intensity}")

    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

# Print the global min and max intensity values from all images
print("\n🌍 Global Intensity Summary Across All Images:")
print(f"   🔽 Global Min Intensity: {global_min_intensity}")
print(f"   🔼 Global Max Intensity: {global_max_intensity}")



🔹 Processing file: patient001_frame01_gt.nii.gz
   📏 Spacing: (1.5625, 1.5625, 10.0)
   📐 Shape: (216, 256, 10) (width, height, depth)
   📦 Image array shape: (10, 256, 216) (depth, height, width)
   🔽 Min intensity value: 0
   🔼 Max intensity value: 3

🌍 Global Intensity Summary Across All Images:
   🔽 Global Min Intensity: 0
   🔼 Global Max Intensity: 3


NiftiImageIO (0x7fae038bf7f0): /Users/muhammadsohaib/code/nii_render/patient001_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x7fae038bf7f0): /Users/muhammadsohaib/code/nii_render/patient001_frame01_gt.nii.gz has unexpected scales in sform



In [1]:
import os
import numpy as np
import SimpleITK as sitk

# Use current working directory
directory_path = os.getcwd()

# Get all NIfTI files in the directory
nii_files = [f for f in os.listdir(directory_path) if f.endswith(".nii.gz")]

# Desired isotropic spacing
iso_spacing = [1.0, 1.0, 1.0]

# Initialize global min and max intensity values
global_min_intensity = float('inf')
global_max_intensity = float('-inf')

def resample_to_isotropic(image, new_spacing, is_label=False):
    original_spacing = image.GetSpacing()
    original_size = image.GetSize()
    
    new_size = [
        int(round(osz * ospc / nspc)) 
        for osz, ospc, nspc in zip(original_size, original_spacing, new_spacing)
    ]
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(new_spacing)
    resampler.SetSize(new_size)
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputDirection(image.GetDirection())
    
    # Choose interpolator
    interpolator = sitk.sitkNearestNeighbor if is_label else sitk.sitkLinear
    resampler.SetInterpolator(interpolator)

    return resampler.Execute(image)

# Process and save isotropic images
for file in nii_files:
    try:
        print(f"\n🔹 Processing file: {file}")
        full_path = os.path.join(directory_path, file)

        # Read image
        image = sitk.ReadImage(full_path)

        # Detect if it's a label (optional: based on filename or logic)
        is_label = any(x in file.lower() for x in ["label", "mask", "gt", "seg"])

        # Resample to isotropic
        iso_image = resample_to_isotropic(image, iso_spacing, is_label=is_label)

        # Save the isotropic image
        output_path = os.path.join(directory_path, f"isotropic_{file}")
        sitk.WriteImage(iso_image, output_path)
        print(f"   💾 Saved isotropic image: {output_path}")

        # Analyze image
        image_array = sitk.GetArrayFromImage(iso_image)
        min_intensity = np.min(image_array)
        max_intensity = np.max(image_array)
        global_min_intensity = min(global_min_intensity, min_intensity)
        global_max_intensity = max(global_max_intensity, max_intensity)

        # Info
        print(f"   📏 Spacing: {iso_image.GetSpacing()}")
        print(f"   📐 Size: {iso_image.GetSize()}")
        print(f"   📦 Shape: {image_array.shape}")
        print(f"   🔽 Min: {min_intensity}")
        print(f"   🔼 Max: {max_intensity}")

    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

# Global summary
print("\n🌍 Global Intensity Summary Across All Resampled Images:")
print(f"   🔽 Global Min Intensity: {global_min_intensity}")
print(f"   🔼 Global Max Intensity: {global_max_intensity}")



🔹 Processing file: isotropic_patient001_frame01_gt.nii.gz
   💾 Saved isotropic image: /Users/muhammadsohaib/code/nii_render/isotropic_isotropic_patient001_frame01_gt.nii.gz
   📏 Spacing: (1.0, 1.0, 1.0)
   📐 Size: (338, 400, 100)
   📦 Shape: (100, 400, 338)
   🔽 Min: 0
   🔼 Max: 3

🔹 Processing file: patient001_frame12.nii.gz


NiftiImageIO (0x7fbfe19c57a0): /Users/muhammadsohaib/code/nii_render/patient001_frame12.nii.gz has unexpected scales in sform

NiftiImageIO (0x7fbfe19c57a0): /Users/muhammadsohaib/code/nii_render/patient001_frame12.nii.gz has unexpected scales in sform



   💾 Saved isotropic image: /Users/muhammadsohaib/code/nii_render/isotropic_patient001_frame12.nii.gz
   📏 Spacing: (1.0, 1.0, 1.0)
   📐 Size: (338, 400, 100)
   📦 Shape: (100, 400, 338)
   🔽 Min: 0
   🔼 Max: 872

🔹 Processing file: patient001_frame12_gt.nii.gz
   💾 Saved isotropic image: /Users/muhammadsohaib/code/nii_render/isotropic_patient001_frame12_gt.nii.gz
   📏 Spacing: (1.0, 1.0, 1.0)
   📐 Size: (338, 400, 100)
   📦 Shape: (100, 400, 338)
   🔽 Min: 0
   🔼 Max: 3

🔹 Processing file: patient001_frame01_gt.nii.gz
   💾 Saved isotropic image: /Users/muhammadsohaib/code/nii_render/isotropic_patient001_frame01_gt.nii.gz
   📏 Spacing: (1.0, 1.0, 1.0)
   📐 Size: (338, 400, 100)
   📦 Shape: (100, 400, 338)
   🔽 Min: 0
   🔼 Max: 3

🌍 Global Intensity Summary Across All Resampled Images:
   🔽 Global Min Intensity: 0
   🔼 Global Max Intensity: 872


NiftiImageIO (0x7fc010fd7190): /Users/muhammadsohaib/code/nii_render/patient001_frame12_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x7fc010fd7190): /Users/muhammadsohaib/code/nii_render/patient001_frame12_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x7fbfe19c5400): /Users/muhammadsohaib/code/nii_render/patient001_frame01_gt.nii.gz has unexpected scales in sform

NiftiImageIO (0x7fbfe19c5400): /Users/muhammadsohaib/code/nii_render/patient001_frame01_gt.nii.gz has unexpected scales in sform

